In [15]:
#!/usr/bin/env python
import sys
import cProfile
import re
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import glob
from netCDF4 import Dataset
from rca_calculate_hdf5_func import rca_calculate_hdf5_func

In [18]:


#if __name__ == "__main__":
#    if len(sys.argv) < 6:
#        print(
#            "ERROR: Arguments are PPI path, date (YYYYMMDD), baseline netCDF path, csv file path, choose use of UZ or Z"
#        )
#        sys.exit(0)

#    datadir = sys.argv[1]
#    date = sys.argv[2]
#    baselinedir = sys.argv[3]
#    csvdir = sys.argv[4]
#    defaultZ = sys.argv[5]
#    print(datadir, date, baselinedir, csvdir, defaultZ)

    # Specify path for that day (datadir)
    # datadir should be input upon running script (sys.argv[1])
datadir = '/run/media/hunzinger/WININSTALL/csapr2-data/'

    # Specify date of interest (date)
    # date ('YYYYMMDD') should be input upon running script (sys.argv[2])
date = '20181106'
yr = date[0:4]
mon = date[4:6]
day = date[6:]

    # Specify directory where the baseline netCDF is located (baselinedir)
    # baselinedir should be input upon running script (sys.argv[3])
baselinedir = '/home/hunzinger/data/'

    # Specify the path to access and write to the running CSV file (csvdir)
    # csvdir should be input upon running script (sys.argv[4])
    # csv_output_path = '/home/hunzinger/data/'
csvdir = '/home/hunzinger/projects/taranis/taranis/calibration/'
csv_filepath = csvdir + "rca_daily_values.csv"

    # Specify the use of UZ or Z in the RCA calculation
    # defaultZ should be input upon running script (sys.argv[5])
defaultZ = 'UZ'
if defaultZ == "UZ":
    uncorrectedZ = True
    dataset = Dataset(baselinedir + "rca_baseline_20181108.nc")
    hPCT_on_50 = dataset.variables["clutter_gate_values_uzh"][:, :]
    vPCT_on_50 = dataset.variables["clutter_gate_values_uzv"][:, :]
    #mhPCT_on_50 = dataset.variables["clutter_gate_mask_uzh"][:, :]
    #mvPCT_on_50 = dataset.variables["clutter_gate_mask_uzv"][:, :]
    hdbz95_baseline = dataset.variables["baseline_dbz95_uzh"][:]
    vdbz95_baseline = dataset.variables["baseline_dbz95_uzv"][:]
    dataset.close()
elif defaultZ == "Z":
    uncorrectedZ = False
    dataset = Dataset(baselinedir + "rca_baseline_20181108.nc")
    hPCT_on_50 = dataset.variables["clutter_gate_values_zh"][:, :]
    vPCT_on_50 = dataset.variables["clutter_gate_values_zv"][:, :]
    hdbz95_baseline = dataset.variables["baseline_dbz95_zh"][:]
    vdbz95_baseline = dataset.variables["baseline_dbz95_zv"][:]
    dataset.close()
else:
    print("ERROR: Must specify the use of UZ or Z in the RCA calculation")

    # Create empty lists to fill using function
dt = []
hdbz95 = []
vdbz95 = []

for f in glob.glob(os.path.join(datadir, "corcsaprM1*.cor-ppi*" + date + "1615*.h5")):
    print(f)
    try:
        DateTime, HDBZ95, VDBZ95 = rca_calculate_hdf5_func(
        f, hPCT_on_50, vPCT_on_50, uncorrectedZ=uncorrectedZ
        )
        # Put all PPI times into a list
        dt.append(DateTime)
        hdbz95.append(HDBZ95)
        vdbz95.append(VDBZ95)
    except OSError:
        pass
    
cProfile.run('re.compile("rca_calculate_hdf5_func(f,hPCT_on_50,vPCT_on_50,uncorrectedZ=uncorrectedZ)")')




/run/media/hunzinger/WININSTALL/csapr2-data/corcsaprM1.00.20181106.181001.raw.cor-ppi-cacti-15tilt_CSAPR2_20181106161503_00.h5
         781 function calls (776 primitive calls) in 0.002 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.002    0.002 <string>:1(<module>)
        4    0.000    0.000    0.000    0.000 enum.py:265(__call__)
        4    0.000    0.000    0.000    0.000 enum.py:515(__new__)
        2    0.000    0.000    0.000    0.000 enum.py:801(__and__)
        1    0.000    0.000    0.002    0.002 re.py:231(compile)
        1    0.000    0.000    0.002    0.002 re.py:286(_compile)
        1    0.000    0.000    0.000    0.000 sre_compile.py:393(_generate_overlap_table)
      2/1    0.000    0.000    0.000    0.000 sre_compile.py:414(_get_literal_prefix)
        1    0.000    0.000    0.000    0.000 sre_compile.py:482(_compile_info)
        2    0.000    0.000    0.000    0.000 s

In [ ]:
    # Calculate mean dBZ95 using all PPI times to get daily mean (H)
    # and calculate RCA using baseline dBZ95
hdbz95_mean = np.nanmean(hdbz95)
hrca_mean = hdbz95_baseline[0] - hdbz95_mean

    # Calculate mean dBZ95 using all PPI times to get daily mean (V)
    # and calculate RCA using baseline dBZ95
vdbz95_mean = np.nanmean(vdbz95)
vrca_mean = vdbz95_baseline[0] - vdbz95_mean

#mhdbz95_mean = np.nanmean(mhdbz95)
#mhrca_mean = hdbz95_baseline[0] - mhdbz95_mean

    # Write daily RCA to CSV file

base = 0  # set to 0 for daily RCA, set to 1 when calculating for baseline
date = yr + "-" + mon + "-" + day

    # Create dictionary and dataframe
csv_frame = pd.read_csv(csv_filepath)
#import pdb; pdb.set_trace()

rca_dict = {
    "DATE": date, "RCA_H": hrca_mean, "RCA_V": vrca_mean, "BASELINE": base
    }
csv_frame = csv_frame.append(rca_dict, ignore_index=True)
csv_frame.set_index('DATE')


csv_frame.to_csv(csv_filepath , index=False)

In [9]:
print(hdbz95_baseline[0])
print(hdbz95)
print(hdbz95_mean)

#print(mhdbz95)
#print(mhdbz95_mean)

print(vdbz95_baseline[0])
#print(vdbz95)
print(vdbz95_mean)


57.52864113984983
[59.0959508184644, 59.032610880415405, 59.396352534268786, 59.4348561886084]
59.239942605439246
58.33197983479645
59.90342827663648
